In [1]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from perlin_noise import PerlinNoise
import netCDF4
from IPython.display import clear_output
import torch
import torch.nn as nn
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

%matplotlib inline

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
def print_peak_gpu_memory():
    peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)  # in MB
    print(f"Peak GPU memory used: {peak_memory:.2f} MB.")
    torch.cuda.reset_peak_memory_stats()  # Reset after printing


def print_gpu_utilization():
    nvmlInit()  # Initialize NVML
    handle = nvmlDeviceGetHandleByIndex(0)  # Assuming we're using GPU 0
    info = nvmlDeviceGetMemoryInfo(handle)  # Get memory info
    print(f"GPU memory occupied: {info.used // 1024**2} MB.")

def visualize(Z_surf,time,H_ice,Lx,Ly):
        clear_output(wait=True)  # Clear the previous output in the notebook
        plt.figure(2, figsize=(11, 4), dpi=200)
        # First subplot: Ice surface
        plt.subplot(1, 2, 1)
        plt.imshow(Z_surf.cpu().numpy(), extent=[0, Lx/1000, 0, Ly/1000], cmap='terrain', origin='lower')
        plt.colorbar(label='Elevation (m)')
        plt.title('Ice Surface at ' + str(int(time)) + ' y')
        plt.xlabel('Distance, km')
        plt.ylabel('Distance, km')
        # Second subplot: Ice thickness
        plt.subplot(1, 2, 2)
        plt.imshow(np.where(H_ice.cpu().numpy() > 0, H_ice.cpu().numpy(), np.nan), extent=[0, Lx/1000, 0, Ly/1000], cmap='jet', origin='lower')
        plt.colorbar(label='Ice Thickness (m)')
        plt.title('Ice Thickness at ' + str(int(time)) + ' y')
        plt.xlabel('Distance, km')
        plt.ylabel('Distance, km')
        # Show the plot
        plt.show()

# Function to create and plot a 2D smooth random ELA field
def generate_and_plot_ELA_field(Z_topo, Z_ELA, ELA_range=200, octaves=4, seed=42, device='cpu'):
    # Get the dimensions of Z_topo
    height, width = Z_topo.shape

    # Initialize Perlin noise
    noise = PerlinNoise(octaves=octaves, seed=seed)

    # Generate a 2D noise field
    noise_field = torch.tensor(
        [[noise([i / height, j / width]) for j in range(width)] for i in range(height)],
        dtype=torch.float32, device=device
    )

    # Normalize the noise field to be between -1 and 1
    min_noise = torch.min(noise_field)
    max_noise = torch.max(noise_field)
    normalized_noise = 2 * (noise_field - min_noise) / (max_noise - min_noise) - 1

    # Scale the noise to the range [Z_ELA - ELA_range, Z_ELA + ELA_range]
    ELA_field = Z_ELA + ELA_range * normalized_noise

    # # Plot the ELA field
    # plt.figure(figsize=(10, 8))
    # plt.imshow(ELA_field.cpu(), cmap='viridis', origin='lower')
    # plt.colorbar(label='ELA (m)')
    # plt.title('2D Smooth Random ELA Field')
    # plt.xlabel('X Coordinate')
    # plt.ylabel('Y Coordinate')
    # plt.show()

    return ELA_field

# Prepare hooks for tensors 
def print_hook_b(grad):
    print("\n db/dELA:",torch.mean(grad))
    return grad
def print_hook_h(grad):
    print("\n dH/db:",torch.mean(grad))
    return grad
def sqrt_hook(grad):
    if torch.abs(torch.mean(grad))>1:
        print(grad**0.5)
        return grad**0.5
def reduce_hook(grad):
    return grad*0.9

In [4]:
# Physical parameters

nc_file = netCDF4.Dataset('bedrock.nc')
Z_topo = torch.tensor(nc_file.variables['topg'][:], device=device, dtype=torch.float32)
ttot = 425  # Time limit (yr)
grad_b = 0.001  # Mass balance gradient
b_max = 0.5  # Maximum precip (m/yr)
# ELA = 3000  # Elevation of equilibrium line altitude (m)
# Generate and plot the ELA field
Z_ELA =1800 #generate_and_plot_ELA_field(Z_topo, ELA,device=device)
rho, g, fd = 910.0, 9.81, 1e-18


In [5]:
def solve_glacier_dynamics(Z_topo, ttot, grad_b, b_max, Z_ELA, rho=rho, g=g, fd=fd,Lx=902000, Ly=602000, dx=2000, dy=2000,dtmax = 1,device=device):
    """
    Solve the glacier dynamics using a diffusion-based solver with PyTorch.
    """
    # Physical parameters (unchanged)

    nx = int(Lx / dx)
    ny = int(Ly / dy)
    epsilon= 1.e-40

    # Initialize ice thickness and surface
    H_ice = torch.zeros((ny, nx), device=device, dtype=torch.float32)
    Z_surf = Z_topo + H_ice
    # Initialize time and iteration counter
    time = torch.tensor(0.0, device=device, dtype=torch.float32)
    dt = torch.tensor(dtmax, device=device, dtype=torch.float32)
    it = 0

    while time < ttot:
        time += dt
        it += 1

        # Compute H_avg
        H_avg = 0.25 * (H_ice[:-1, :-1] + H_ice[1:, 1:] + H_ice[:-1, 1:] + H_ice[1:, :-1])

        # Compute Snorm
        Sx = (Z_surf[:, 1:] - Z_surf[:, :-1]) / dx
        Sy = (Z_surf[1:, :] - Z_surf[:-1, :]) / dy
        Sx = 0.5 * (Sx[:-1, :] + Sx[1:, :])
        Sy = 0.5 * (Sy[:, :-1] + Sy[:, 1:])
        Snorm = torch.sqrt(Sx**2 + Sy**2 + epsilon)  # Add a small value to avoid division by zero

        # Compute diffusivity
        D = fd * (rho * g)**3.0 * H_avg**5 * Snorm**2+ epsilon

        # Compute adaptive time step
        dt = min(min(dx, dy)**2 / (4.1 * torch.max(D).item()), dtmax)

        # Compute fluxes
        qx = -(0.5 * (D[:-1, :] + D[1:, :])) * (Z_surf[1:-1, 1:] - Z_surf[1:-1, :-1]) / dx
        qy = -(0.5 * (D[:, :-1] + D[:, 1:])) * (Z_surf[1:, 1:-1] - Z_surf[:-1, 1:-1]) / dy

        # Compute thickness change rate
        dHdt = -(torch.diff(qx, dim=1) / dx + torch.diff(qy, dim=0) / dy)

        # Update ice thickness
        H_ice[1:-1, 1:-1] += dt * dHdt

        # Compute surface mass balance (SMB)
        b = torch.minimum(grad_b * (Z_surf - Z_ELA), torch.tensor(b_max, device=device))
        b.retain_grad() 
        b.register_hook(reduce_hook)

        H_ice[1:-1, 1:-1] += dt * b[1:-1, 1:-1]

        # Ensure ice thickness remains positive
        H_ice = torch.maximum(H_ice, torch.tensor(0.0, device=device))
        # Update surface topography
        Z_surf = Z_topo + H_ice

        # Visualization at specified intervals
        # if it % 50 == 0:
        #     visualize(Z_surf,time,H_ice,Lx,Ly)
    return H_ice, it


In [6]:
Z_ELA = torch.full(Z_topo.shape,2000., requires_grad=True, device=device)
h,it=solve_glacier_dynamics(Z_topo, ttot, grad_b, b_max, Z_ELA)
print(it)
# torch.save(h,'Obs_2D.pt')

1469


In [ ]:
print_gpu_utilization()
print_peak_gpu_memory()

### Inversion with ADAM

In [ ]:
# Reset the starting point in tracking maximum GPU memory occupied by tensors in bytes for a given device.
torch.cuda.reset_peak_memory_stats()
# Initial guesses for inversion problem
Z_ELA = torch.full(Z_topo.shape,2200., requires_grad=True, device=device)

# Observed glacier thickness (assumed already loaded as observed_thk tensor)
observed_thk = torch.load('Obs_2D.pt', weights_only=True).to(device) # Ensure it's on the right device

# Define initial and final learning rates
initial_lr = 7
final_lr = 5
reg_lambda=0.000005
n_iterations = 10

# Optimizer setup
optimizer = torch.optim.Adam([Z_ELA], lr=initial_lr)

# Initialize lists to track loss components
total_loss_history = []
data_fidelity_history = []
regularization_history = []
total_gradients_history=[]

for i in range(n_iterations):
    # Update the learning rate
    lr = initial_lr - (i / (n_iterations - 1)) * (initial_lr - final_lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    optimizer.zero_grad()  # Zero gradients
    
    # Perform forward simulation
    H_simulated, it= solve_glacier_dynamics(Z_topo, ttot, grad_b, b_max, Z_ELA)

    print(it)
    # Compute data fidelity term
    data_fidelity = torch.mean((H_simulated - observed_thk) ** 2)

    # Compute smoothness regularization (differences in x and y directions)
    smoothness_x = torch.sum((Z_ELA[:, 1:] - Z_ELA[:, :-1]) ** 2)
    smoothness_y = torch.sum((Z_ELA[1:, :] - Z_ELA[:-1, :]) ** 2)
    smoothness_reg = smoothness_x + smoothness_y

    # Total loss
    loss = data_fidelity + reg_lambda * smoothness_reg
     
    # Backpropagate loss and update parameters
    loss.backward()

    # Store the gradients and ELA evolution
    total_gradients_history.append(torch.max(Z_ELA.grad).item())

    optimizer.step()

    # Store loss components for plotting later
    total_loss_history.append(loss.item())
    data_fidelity_history.append(data_fidelity.item())
    regularization_history.append((reg_lambda * smoothness_reg).item())
    # Print loss, gradients and current parameters every 50 iterations 
    if (i + 1) % 2 == 0:
        print(f"\nIteration {i+1}/{n_iterations}, Loss: {loss:.3f}, ELA: {torch.mean(Z_ELA).item()}; {torch.min(Z_ELA).item()}")
        print(f"Gradient of ELA : {torch.max(Z_ELA.grad).item()}")

        print_gpu_utilization()  # Print memory after the loop
        print_peak_gpu_memory()  # Print the peak memory   

In [ ]:
plt.figure(figsize=(10,6))
# plt.plot(ELA_evolution,label="evolution of ELA",color='b')
plt.plot(total_gradients_history, label='Evolution of gradients')
plt.xlabel('Iteration')
plt.legend()
plt.show()

In [ ]:
# Function to plot the loss function components
def plot_loss_components(total_loss_history, data_fidelity_history, regularization_history):
    plt.figure(figsize=(10, 6))

    # Plot the total loss, data fidelity, and regularization term
    plt.plot(total_loss_history, label='Total Loss', color='b', linewidth=2)
    plt.plot(data_fidelity_history, label='Data Fidelity', color='g', linestyle='--', linewidth=2)
    plt.plot(regularization_history, label='Regularization (Smoothness)', color='r', linestyle='-.', linewidth=2)

    # Add labels and legend
    plt.xlabel('Iteration')
    plt.ylabel('Loss Value')
    plt.title('Loss Function Components Over Iterations')
    plt.legend()

    # Show the plot
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_loss_components(total_loss_history, data_fidelity_history, regularization_history)

In [ ]:
# Plot the ELA field
plt.figure(figsize=(10, 8))
# Detach the tensor from the computation graph and move it to the CPU before converting to a NumPy array
plt.imshow(Z_ELA.detach().cpu().numpy(), cmap='viridis', origin='lower')
plt.colorbar(label='ELA (m)')
plt.title('Reconstructed ELA Field')
plt.savefig('inverted_ELA')
plt.show()


In [ ]:
# Save ELA
torch.save(Z_ELA,"inverted_ELA.pt")